# Auxiliar 1 


----------------------------------

## 📚 Objetivos de la clase 📚

El objetivo principal de esta clase es introducirlos a la clasificación de texto en NLP. 
Para esto, implementaremos varios modelos de clasificación destinados a **predecir la categoría de noticias de la radio biobio**.

Los modelos y métodos que usaremos serán las vistas en las clases anteriores: 

- Tokenización, Stemming, Lematización y eliminación de Stop Words.
- Bag of Words.
- Claisifcador de Bayes .
- Logistic regression.

La clase estará enfocada en utilizar las siguientes librerías (muy utilizadas en NLP):

- Pandas
- Scikit-Learn
- Spacy
- NLTK

Una vez resuelto, pueden utilizar cualquier parte del código que les parezca prudente para la tarea 1 (que también es de clasificación de texto! 😊).


El notebook del auxiliar ya ejecutado se encuentra tanto en el [github](https://github.com/dccuchile/CC6205/tree/master/tutorials) del curso (Recuerden dejar su Star ⭐😉!), como en un colab de google. 

Para correr localmente el auxiliar, se recomienda instalar todos los paquetes usando anaconda,

## Importar las librerías

### En local: Python y Conda

Primero, si es que no tienen aun las librerías, hay que instalarlas.
Recuerden que usaremos `python 3.7` junto a `conda` como gestor de paquetes para el curso.

Este pueden descargarlo e instalenlo desde aquí : [🐍 Anaconda 🐍](https://www.anaconda.com/distribution/).

Para instalar las librerías, ejecutar en una consola 💻:

```cmd
conda install pandas scikit-learn ntkl spacy 
```
Y luego descargar el modelo de spacy en español: 

```cmd
python -m spacy download es_core_news_sm
```

Si saben un poco mas de anaconda, pueden instalar sus paquetes en un ambiente exlcusivo para el curso. Pero no es necesario!! Mas información [aquí](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).

### En colab

TODO


### Importar

In [8]:
import pandas as pd    
import spacy
import nltk

In [9]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
from nltk.stem import SnowballStemmer
from spacy.lang.es.stop_words import STOP_WORDS
nlp = spacy.load("es_core_news_sm", disable=['ner', 'parser', 'tagger'])
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Clasificación de Texto 

¿Cuál de estos emails es SPAM?

Respuesta:

Pero, también pueden representar otras clases...

¿Cómo habrán encontrado la película?

Entonces, qué es la clasificación de texto?

## Preprocesamiento del texto

En esta sección, cargaremos y los procesaremos el dataset de textos con el fin de representar de buena manera los datos para las tareas que debemos realizar. En esta, utilizaremos las siguientes técnicas: 

- Tokenización
- Eliminación de Stopwords
- Stemming
- Lematización
- Transformación de Tokens a Bag of Words

### Cargar los datasets 



Los datos que usaremos son 5 conjuntos de noticias extraidos de la radio biobio.
Cada categoría contiene 200 documentos (noticias). 

Los cargaremos utilizando la librería pandas: 


In [ ]:
nacional = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_nacional.json", encoding ='utf-8')
internacional = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_internacional.json", encoding ='utf-8')
economia = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_economia.json", encoding ='utf-8')
sociedad = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_sociedad.json", encoding ='utf-8')
opinion = pd.read_json("https://github.com/dccuchile/CC6205/releases/download/Data/biobio_opinion.json", encoding ='utf-8')

datasets = [nacional, internacional, economia, sociedad, opinion]
dataset = pd.concat(datasets)

#### Vizualizar que es lo que cargamos

In [ ]:
# Por ejemplo, examinemos las columnas del dataset de noticias de la categoría sociedad 
# (Todos los datasets tienen el mismo formato)
sociedad.describe()

#### Ejemplo de noticia de categoría sociedad: 

In [ ]:
# Extraigamos una noticia de ejemplo desde el dataset
sample = sociedad.iloc[19:20]

In [ ]:
sample

In [ ]:
# Extraemos el contenido y la categoría de la noticia seleccionada.
sample_content = sample.content.values[0]
sample_category = sample.category.values[0]

In [ ]:
print("Contenido:\n\n", sample_content.strip(),
      "\n\nClase Correspondiente:\n\n", sample_category)

### Tokenizar

¿Qué era tokenizar?

    In computer science, lexical analysis, lexing or tokenization is the process of converting a sequence of characters (such as in a computer program or web page) into a sequence of tokens (strings with an assigned and thus identified meaning).
    
Referencia: [Tokenización en wikipedia](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)



#### spaCy y el objeto nlp

`nlp` es el objeto que nos permite usar e interactuar con la librería [`spacy`](https://spacy.io/).
Esta librería incluye variadas herramientras, tales como tokenizar, lematizar, descartar stopwords, entre otras (para este auxiliar, solo utilizaremos las mencionadas). El objeto nlp lo instanciamos en la sección de imports.

Para usarla, simplemente se le pasa el texto como parámetro, como veremos en el siguiente ejemplo: 

In [ ]:
for word in nlp("hola gentíl ciudadano, ¿qué tal?"):
    print(word)

#### Tokenizemos la noticia de ejemplo: 

In [ ]:
tokenized_content = [word.text for word in nlp(sample_content)]

Observación: 

    tokenized_content = [word.text for word in nlp(sample_content)] 

Equivale a :

    tokenized_content = []
    for word in nlp(sample_content):
        tokenized_content.append(word.text)



Examinemos como quedó el texto tokenizado:

In [ ]:
pd.DataFrame(tokenized_content)[0:10]

Que es básicamente, un arreglo (o vector) con las palabras:

In [ ]:
tokenized_content[1:10]

### Stopwords 

¿Qué eran las stopwords?

    In computing, stop words are words which are filtered out before or after processing of natural language data (text).[1] Stop words are generally the most common words in a language, there is no single universal list of stop words used by all natural language processing tools, and indeed not all tools even use such a list. Some tools avoid removing stop words to support phrase search. 
    
Referencias: [Stopwords en Wikipedia](https://en.wikipedia.org/wiki/Stop_words)

En este caso, utilizaremos las stopwords inlcuidas en la librería spaCy en español

In [ ]:
len(STOP_WORDS)

In [ ]:
pd.DataFrame(STOP_WORDS).sample(10)

#### Remover las stopwords

In [ ]:
tokenized_content_no_stop_words = [
    token for token in tokenized_content if token not in STOP_WORDS
]

In [ ]:
pd.DataFrame(zip(tokenized_content, tokenized_content_no_stop_words),
             columns=['original', 'no stopwords'])[1:10]

### Stemming

¿Qué era el stemming? 

    Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form—generally a written word form.
    
Referencia: [Stemming en Wikipedia](https://en.wikipedia.org/wiki/Stemming)
  
#### Ejemplos: 


| word | stem of the word  |
|---|---|
working | work
worked | work
works | work

#### nltk

En este caso, utilizaremos la segunda librería de herramientas de nlp: [`nltk`](https://www.nltk.org/). Esta provee una buena herramienta para hacer stemming en español : `SnowballStemmer`

In [ ]:
stemmer = SnowballStemmer('spanish')
stemmed_content = [stemmer.stem(word) for word in tokenized_content]

In [ ]:
pd.DataFrame(zip(tokenized_content, stemmed_content),
             columns=['original', 'stem'])[1:10]

### Lematización

¿Qué era lematización? 

    
    Lemmatisation (or lemmatization) in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.[
    
    
Referencia: [Lematización en wikipedia](https://en.wikipedia.org/wiki/Lemmatisation)
    
#### Ejemplos

| word | lemma  |
|---|---|
dije| decir 
guapas | guapo
mesa | mesas


#### Lematizar el texto

Al igual que la tokenización, utilizaremos `scpaCy` (a través del objeto `nlp`) para lematizar el contenido.

In [ ]:
lemmatized_content = [word.lemma_ for word in nlp(sample_content)]

In [ ]:
# Visualizar la lematización
pd.DataFrame(zip(tokenized_content, lemmatized_content),
             columns=['original', 'lemma'])[1:10]

### Comparativa entre stemming y lematizar

Discusión: 

    ¿Cuál es mejor?

In [ ]:
pd.DataFrame(zip(tokenized_content, stemmed_content, lemmatized_content),
             columns=['original', 'stem', 'lemma'])[1:15]

### Bag of Words (BoW)

¿Qué es?


    
    The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as the bag (multiset) of its words 
    
Referencia: [BoW en wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)

### Ejemplo

- Doc1 : 'I love dogs'
- Doc2: 'I hate dogs and knitting.
- Doc3: 'Knitting is my hobby and my passion.

![BOW](https://i1.wp.com/datameetsmedia.com/wp-content/uploads/2017/05/bagofwords.004.jpeg)

### scikit-learn y CountVectorizer

Para transformar los tokens al modelo `bag of words (BoW)` usaremos la librería [`scikit-learn`](https://scikit-learn.org/stable/), específicamente, la clase [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

Esta requiere que definamos tokenizadores. Es decir, funciones que tomen un documento y lo retornen como una lista de palabras. 

En los tokenizadores podemos definir si dejar o quitar stopwords, lematizar, hacer stemming, entre otras.

#### Tokenizadores para CountVectorizer

In [ ]:
# Tokenizers for CountVectorizer


# Solo tokenizar el doc.
def tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]


# Tokenizar y remover las stopwords del doc
def tokenizer_with_stopwords(doc):
    return [x.orth_ for x in nlp(doc) if x.orth_ not in STOP_WORDS]


# Tokenizar y lematizar.
def tokenizer_with_lemmatization(doc):
    return [x.lemma_ for x in nlp(doc)]


# Tokenizar y hacer stemming.
def tokenizer_with_stemming(doc):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in [x.orth_ for x in nlp(doc)]]

In [ ]:
# Instanciamos CountVectorizer con el tokenizador seleccionado. 
# Definimos si queremos n-gramas en el último parámetro.
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=tokenizer,
                             ngram_range=(1, 1))

# Extraemos 4 noticias desde opinion y se la entregamos al vectorizador para que las transforme a vectores BoW.
bow = vectorizer.fit_transform(opinion.sample(4).content)

# Examinamos el primero
bow[0]

In [ ]:
#vectorizer.vocabulary_

In [ ]:
# Visualizemos el BoW generado para este caso
bow[0].toarray()[0][0:10]

## Clasificación de Texto

### ¿En qué consiste la clasificación de texto?

Según [wikipedia](https://en.wikipedia.org/wiki/Document_classification): 

    "The task is to assign a document to one or more classes or categories"


Algunos ejemplos:

- Assigning subject categories, **topics**, or genres
- Spam detection 
- Authorship identification
- Age/gender identification
- Language Identification
- Sentiment analysis
- ...

### Definición formal

Input:
- A document    $d$
- A fixed set of classes $C=\{c_{1},    c_{2},...,    c_{J}\}$

Output:    

- A predicted class $c \in C$

### Tipos de técnicas de clasificación: 

- Hand-coded Rules. (No se verán en este aux).
- Supervised Machine Learning: 
    - Naïve Bayes
    - Logistic regression
    - Support vector machines
    - Muchos mas

### Procesar los datasets

Seleccionar solo las columnas relevantes y divider en conjuntos de entrenamiento y de prueba.

In [ ]:
def process_datasets(dataset):
    X_train, X_test, y_train, y_test = train_test_split(dataset.content, dataset.category, test_size=0.33, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = process_datasets(dataset)

### Clasificación de tópicos de noticias con Naive Bayes


¿Qué es?

    In machine learning, naive Bayes classifiers are a family of simple "probabilistic classifiers" based on applying Bayes' theorem with strong (naive) independence assumptions between the features. 

- Simple (“naïve”) classification method based on Bayes rule that relies on very simple representation of document: *Bag of words*:

Given a problem instance to be classified, represented by a vector $x =(x_{1},\dots ,x_{n})$ representing some n features (independent variables), it assigns to this instance probabilities:

$$ p(C_k | x_1, \dots, x_n) $$

or each of $K$ possible outcomes or classes $ C_{k}$.


Using Bayes' theorem, the conditional probability can be decomposed as 

$$ p(C_k | x ) = \frac{p (C_k) p(x | C_k)}{p(x)}$$

In plain english:

$$posterior = \frac{prior * likehood}{evidence} $$


#### En este caso...

$$ p(\ nacional\ |\  [0,0,3,0,6,0,2,\dots] ) = \frac{p (\ nacional\ )\ p([0,0,3,0,6,0,2,\dots]\ | \ nacional)}{p([0,0,3,0,6,0,2,\dots])}$$


Referencia : [Naive Bayes Classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)

Observación: Dado el supuesto de independencia del clasificador, no se deben utilizar n-gramas.


#### Establecer el Pipeline


Un [`pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) es la definición del proceso que llevará a cabo el programa para preprocesarlo y despues clasificarlo. 
Puede tener múltiples etapas.

In [ ]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word', tokenizer = TOKENIZER, ngram_range=(1,1))  

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Definimos el pipeline
text_clf = Pipeline([('vect', vectorizer), ('clf', clf)])

#### Entrenar

Entrenamos el nuevo clasificador

In [ ]:
text_clf.fit(X_train, y_train)

#### Evaluar

Evaluamos el rendimiento del clasificador que acabamos de entrenar, a traves de:

- [`Matriz de confusión`](https://es.wikipedia.org/wiki/Matriz_de_confusi%C3%B3n)
- [`Indice F1`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) (Puntaje, mientras mas alto mejor)

En la matriz de confusión: 

- `precision` is the number of correct results divided by the number of all returned results. 
- `recall` is the number of correct results divided by the number of results that should have been returned

In [ ]:
predicted = text_clf.predict(X_test)

conf = confusion_matrix(y_test, predicted)
score = f1_score(y_test, predicted, average='macro') 
class_rep = classification_report(y_test, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nF1:'+str(score))


#### Ejemplos

In [ ]:
text_clf.predict(["En puerto montt se encontró un perrito, que aparentemente, habría consumido drogas de alto calibre. Producto de esto, padecera severa caña durante varios dias."])

In [ ]:
text_clf.predict(["kim jong un será el próximo candidato a ministro de educación."])

In [ ]:
text_clf.predict(["El banco mundial presentó para chile un decrecimiento económico de 92% y una inflación de 8239832983289%."])

### Clasificación de tópicos de noticias con Regresión Logísitica

No profundizaremos en este clasificador, mas del hecho de que se "supone" que debería tener mejor rendimiento que el de bayes.

Referencia: [Regresión Logística](https://en.wikipedia.org/wiki/Logistic_regression)

#### Establecer el Pipeline

In [ ]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

#### Entrenar

In [ ]:
log_pipe.fit(X_train, y_train)

#### Evaluar

In [ ]:
predicted = log_pipe.predict(X_test)

conf = confusion_matrix(y_test, predicted)
score = f1_score(y_test, predicted, average='macro') 
class_rep = classification_report(y_test, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nF1 Score: '+str(score))

#### Ejemplos

In [ ]:
log_pipe.predict(["En puerto montt se encontró un perrito, que aparentemente, habría consumido drogas de alto calibre. Producto de esto, padecera severa caña durante varios dias."])

In [ ]:
log_pipe.predict(["kim jong un será el próximo candidato a ministro de educación."])

### Clasificación de Autoría de documentos

¿Existirá un patrón en como escriben los periodistas que nos permitan identificarlos a partir de sus textos?

In [ ]:
pd.concat(datasets).author.unique()

In [ ]:
def process_datasets_by_author(datasets):
    dataset = pd.concat(datasets)
    X_train, X_test, y_train, y_test = train_test_split(dataset.content, dataset.author, test_size=0.33, random_state=42)
    
    return X_train, X_test, y_train, y_test


In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = process_datasets_by_author(datasets)

#### Definir el Pipeline

In [ ]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

log_mod_by_author = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe_by_author = Pipeline([('vect', vectorizer), ('clf', log_mod_by_author)])

#### Entrenar

In [ ]:
log_pipe_by_author.fit(X_train_2, y_train_2)

#### Evaluar

In [ ]:
predicted = log_pipe_by_author.predict(X_test_2)

conf = confusion_matrix(y_test_2, predicted)
score = f1_score(y_test_2, predicted, average='macro') 
class_rep = classification_report(y_test_2, predicted)

print('\nConfusion Matrix for Logistic Regression + ngram features:')
print(conf)
print('\nClassification Report')
print(class_rep)
print('\nF1 Score:'+str(score))

## Créditos

Todas las noticias extraidas perteneces a [Biobio Chile](https://www.biobiochile.cl/), los cuales gentilmente licencian todo su material a través de la [licencia Creative Commons (CC-BY-NC)](https://creativecommons.org/licenses/by-nc/2.0/cl/)

## Referencias

Gitgub del curso: 
- https://github.com/dccuchile/CC6205

Slides:
- https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf


Análisis de sentimientos como clasificación de texto:
- https://affectivetweets.cms.waikato.ac.nz/benchmark/

Algunos Recursos útiles
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [Scikit-learn Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf)
- [Spacy Tutorial](https://www.datacamp.com/community/blog/spacy-cheatsheet)
- [NLTK Cheat sheet](http://sapir.psych.wisc.edu/programming_for_psychologists/cheat_sheets/Text-Analysis-with-NLTK-Cheatsheet.pdf)